In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import requests
from lxml.html import parse

from urllib.request import urlopen
from pandas.io.parsers import TextParser
import struct
print('Libraries imported.')

Libraries imported.


In [2]:
#read the data

criminal = pd.read_csv('KSI_CLEAN.csv')

criminal.head()

,ACCNUM,YEAR,MONTH,DAY,HOUR,MINUTES,WEEKDAY,LATITUDE,LONGITUDE,Ward_Name,Ward_ID,Hood_Name,Hood_ID,Division,District,STREET1,STREET2,OFFSET,ROAD_CLASS,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,FATAL
0,1249781,2011,8,4,23,18,3,43.651545,-79.383490,Toronto Centre-Rosedale (27),27,Bay Street Corridor (76),76,D52,Toronto East York,QUEEN ST W,YORK ST,,Major Arterial,Mid-Block,,No Control,Clear,Dark,Dry,Non-Fatal Injury,Pedestrian Collisions,Driver,50 to 54,None,0,West,Municipal Transit Bus (TTC),Going Ahead,Driving Properly,Normal,,,,,,,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,1311542,2012,8,19,23,18,6,43.780445,-79.300490,Scarborough-Agincourt (40),40,Tam O'Shanter-Sullivan (118),118,D42,Scarborough,SHEPPARD AVE E,AMETHYST RD,,Major Arterial,Intersection,,No Control,Clear,"Dark, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,Passenger,25 to 29,Minimal,0,,Other,,,,,,,,,,1,0,1,0,0,0,0,1,1,1,0,0,0,0
2,5002235651,2015,12,30,23,39,2,43.682342,-79.328266,Toronto-Danforth (30),30,Greenwood-Coxwell (65),65,D55,Toronto East York,DANFORTH AVE,GILLARD AVE,,Major Arterial,Intersection,At Intersection,No Control,Clear,"Dark, artificial",Dry,Fatal,Pedestrian Collisions,Pedestrian,60 to 64,Fatal,64,South,,,,,Vehicle is going straight thru inter.while ped...,"Crossing, no Traffic Control",Normal,,,,1,0,1,0,0,0,0,0,0,1,0,0,0,1
3,1311542,2012,8,19,23,18,6,43.780445,-79.300490,Scarborough-Agincourt (40),40,Tam O'Shanter-Sullivan (118),118,D42,Scarborough,SHEPPARD AVE E,AMETHYST RD,,Major Arterial,Intersection,,No Control,Clear,"Dark, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,Vehicle Owner,unknown,,0,,Other,,,,,,,,,,1,0,1,0,0,0,0,1,1,1,0,0,0,0
4,1311542,2012,8,19,23,18,6,43.780445,-79.300490,Scarborough-Agincourt (40),40,Tam O'Shanter-Sullivan (118),118,D42,Scarborough,SHEPPARD AVE E,AMETHYST RD,,Major Arterial,Intersection,,No Control,Clear,"Dark, artificial",Dry,Non-Fatal Injury,Pedestrian Collisions,Driver,60 to 64,None,0,West,"Automobile, Station Wagon",Going Ahead,Failed to Yield Right of Way,Normal,,,,,,,1,0,1,0,0,0,0,1,1,1,0,0,0,0


In [3]:
# Select related attributes from dataset
criminal = criminal[['ACCNUM','LATITUDE','LONGITUDE','District','FATAL']]
criminal.head()

,ACCNUM,LATITUDE,LONGITUDE,District,FATAL
0,1249781,43.651545,-79.383490,Toronto East York,0
1,1311542,43.780445,-79.300490,Scarborough,0
2,5002235651,43.682342,-79.328266,Toronto East York,1
3,1311542,43.780445,-79.300490,Scarborough,0
4,1311542,43.780445,-79.300490,Scarborough,0


In [4]:
#Add district label 0-3 accordingly
criminal['District Label']=0
district = criminal['District']
district_label =criminal['District Label']
for i in range(len(criminal)):
    if district[i] == 'Toronto East York':
        district_label[i] = '1'
    if district[i] == 'Etobicoke York':
        district_label[i] = '2'
    if  district[i] == 'North York':
        district_label[i] = '3'

In [4]:
criminal.head()

,ACCNUM,LATITUDE,LONGITUDE,District,FATAL,District Label
0,1249781,43.651545,-79.383490,Toronto East York,0,1
1,1311542,43.780445,-79.300490,Scarborough,0,0
2,5002235651,43.682342,-79.328266,Toronto East York,1,1
3,1311542,43.780445,-79.300490,Scarborough,0,0
4,1311542,43.780445,-79.300490,Scarborough,0,0


In [5]:
#take 300 sample to draw a map, fatal accidents are in red

criminalx = criminal.sample(300)
address = 'Toronto, CA'

geolocator = Nominatim(user_agent=None)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
colorx = pd.DataFrame({'color' : ['#d8b634', '#a2756c','#906ca1','#7da36d']})



# create map

kclusters = 4
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster,fatal in zip(criminalx['LATITUDE'], criminalx['LONGITUDE'], criminalx['District Label'],criminalx['FATAL']):
    label = folium.Popup(' District ' + str(cluster)+' fatal ' + str(fatal), parse_html=True)
    color_index = 'red'
    if fatal != 1:
        color_index = colorx['color'][int(cluster)]
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=color_index,
#         color = colorx['color'][int(cluster)],
        fill=True,
        fill_color=color_index,
#         fill_color = colorx['color'][int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [7]:
#see the relationship between fatal and district
criminal.groupby(['FATAL','District']).groups

{(0,
  'Etobicoke York'): Int64Index([    6,     7,    25,    26,    27,    32,    33,    34,    35,
                36,
             ...
             12509, 12510, 12517, 12518, 12519, 12522, 12523, 12533, 12534,
             12535],
            dtype='int64', length=2625),
 (0,
  'North York'): Int64Index([   10,    11,    12,    16,    17,    18,    19,    23,    24,
                40,
             ...
             12470, 12489, 12490, 12491, 12492, 12524, 12527, 12528, 12529,
             12530],
            dtype='int64', length=2017),
 (0,
  'Scarborough'): Int64Index([    1,     3,     4,     5,    20,    21,    42,    43,    76,
                77,
             ...
             12467, 12471, 12472, 12486, 12487, 12488, 12495, 12496, 12536,
             12537],
            dtype='int64', length=2437),
 (0,
  'Toronto East York'): Int64Index([    0,     8,     9,    13,    14,    15,    22,    28,    29,
                55,
             ...
             12514, 12515, 12516, 1252